<a href="https://colab.research.google.com/github/Hawawou/UE-803/blob/main/Part2_Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Verification against KG

In [6]:
import pandas as pd

In [7]:
ne_data = pd.read_csv('named_entities.csv')
ne_data.head()

,Category,Name,Content,spacy_entities,stanza_entities
0,Computer_Scientists,Atta ur Rehman Khan,atta ur rehman khan urdu عطا الرحمن خان comput...,"[('atta ur rehman khan', 'PERSON'), ('عطا', 'O...","[('stanford', 'ORG'), ('2', 'CARDINAL'), ('201..."
1,Computer_Scientists,Wil van der Aalst,willibrordus martinus pancratius van der aalst...,"[('pancratius van der aalst', 'PERSON'), ('29'...","[('29 january 1966', 'DATE'), ('1988', 'DATE')..."
2,Computer_Scientists,Scott Aaronson,scott joel aaronson born may 21 1981 american ...,"[('scott joel aaronson', 'PERSON'), ('21 1981'...","[('scott joel aaronson', 'PERSON'), ('21 1981'..."
3,Computer_Scientists,Rediet Abebe,rediet abebe amharic ረድኤት አበበ ethiopian comput...,"[('rediet abebe', 'PERSON'), ('ረድኤት', 'GPE'), ...","[('abebe', 'PERSON'), ('ethiopian', 'NORP'), (..."
4,Computer_Scientists,Hal Abelson,harold abelson born april 26 1947 american mat...,"[('harold abelson', 'PERSON'), ('april 26 1947...","[('harold abelson', 'PERSON'), ('april 26 1947..."


In [38]:
df = pd.read_csv('rdf_cleaned.csv')
df.head()

,Subject,Predicate,Object
0,Jeff_Bonwick,wikiPageWikiLink,Program_optimization
1,Rod_Brooks,wikiPageWikiLink,Moore_machine
2,Leonard_Adleman,wikiPageWikiLink,Hulk_(film)
3,John_Armleder,wikiPageWikiLink,Kunstmuseum_Den_Haag
4,Tracy_Camp,wikiPageWikiLink,Eldon_C._Hall


In [39]:
# Remove underscores from Subject and Object
df['Subject'] = df['Subject'].apply(lambda x: x.replace('_', ' '))
df['Object'] = df['Object'].str.replace("_", " ")
df.head()

,Subject,Predicate,Object
0,Jeff Bonwick,wikiPageWikiLink,Program optimization
1,Rod Brooks,wikiPageWikiLink,Moore machine
2,Leonard Adleman,wikiPageWikiLink,Hulk (film)
3,John Armleder,wikiPageWikiLink,Kunstmuseum Den Haag
4,Tracy Camp,wikiPageWikiLink,Eldon C. Hall


In [40]:
kg_data = df.groupby('Subject')['Object'].apply(list).to_dict()
kg_data

{'Abed Abdi': ['Osama Hajjaj',
  'John Doyle (Irish artist)',
  'Category:Caricaturists',
  'Thomas Rowlandson',
  'Emad Hajjaj',
  'Glen Hanson',
  'Amnon David Ar',
  'Donald Bevan',
  'S. Jithesh',
  'Jovan Prokopljević',
  'Arthur Good',
  'Dušan Petričić',
  'Sam Viviano',
  'Predrag Koraksić Corax',
  'List of graphic designers',
  'Gerald Scarfe',
  'Pier Leone Ghezzi',
  'Jeff Hook',
  'Murray Webb',
  'Honoré Daumier',
  'Jacques Callot',
  'Melchiorre Delfico (caricaturist)',
  'Drew Friedman (cartoonist)',
  'Émile Cohl',
  'Marjorie Organ',
  'Richard Newton (caricaturist)',
  'Gavin Bell',
  'Boris Yefimov',
  'Max Beerbohm',
  'John Leech (caricaturist)',
  'Thomas Nast',
  'John Tenniel',
  'Jean-Pierre Dantan',
  'Kenny Meadows',
  'Andre Gill',
  'Kate Carew',
  'Bagaria',
  'George Cruikshank',
  'Alfred Grévin',
  'Daniel Stieglitz',
  'George Bahgoury',
  'Steve Bell (cartoonist)',
  'Sam Berman',
  'Aurelius Battaglia',
  'Abed Abdi',
  'Kerry G. Johnson',
  'Herma

In [10]:
import re

In [54]:
def evaluate_ne_to_kg(kg_data, data):
    results = {}
    for index, row in data.iterrows():
        person = row['Name']
        ne_spacy = row['spacy_entities']
        ne_stanza = row['stanza_entities']

       
        kg_entities = kg_data.get(person, [])

        # Initialize counter
        spacy_total = len(ne_spacy)
        spacy_matched = 0

        stanza_total = len(ne_stanza)
        stanza_matched = 0
        
        # Check if any named entity matches a KG entity
        def match_ne_to_kg(ne, kg_entities):
            ne_text, ne_label = ne
            for kg_entity in kg_entities:
                
                # Use re to find matches considering potential variations in text
                pattern = re.escape(ne_text)
                if re.search(pattern, kg_entity, re.IGNORECASE):
                    return True
            return False
        
          # Evaluate NE prediction for stanza
        for ne_stan in ne_stanza:
            if not isinstance(ne_stan, tuple):
                continue
            print(ne_stan)
            if match_ne_to_kg(ne_stan, kg_entities):
                stanza_matched += 1

        # Evaluate NE prediction for spacy
        for ne_spa in ne_spacy:
            if not isinstance(ne_spa, tuple):
                continue
            if match_ne_to_kg(ne_spa, kg_entities):
                spacy_matched += 1

        # Ratio
        spacy_ratio = spacy_matched / spacy_total if spacy_total > 0 else 0
        stanza_ratio = stanza_matched / stanza_total if stanza_total > 0 else 0

        results[person] = {'number_of_spacy_entity': spacy_matched,'number_of_stanza entity': stanza_matched,'spacy_ratio': spacy_ratio, 'stanza_ratio': stanza_ratio}

    return results



In [55]:
matched_entities = evaluate_ne_to_kg(kg_data, ne_data)
matched_entities

{'Atta ur Rehman Khan': {'number_of_spacy_entity': 0,
  'number_of_stanza entity': 0,
  'spacy_ratio': 0.0,
  'stanza_ratio': 0.0},
 'Wil van der Aalst': {'number_of_spacy_entity': 0,
  'number_of_stanza entity': 0,
  'spacy_ratio': 0.0,
  'stanza_ratio': 0.0},
 'Scott Aaronson': {'number_of_spacy_entity': 0,
  'number_of_stanza entity': 0,
  'spacy_ratio': 0.0,
  'stanza_ratio': 0.0},
 'Rediet Abebe': {'number_of_spacy_entity': 0,
  'number_of_stanza entity': 0,
  'spacy_ratio': 0.0,
  'stanza_ratio': 0.0},
 'Hal Abelson': {'number_of_spacy_entity': 0,
  'number_of_stanza entity': 0,
  'spacy_ratio': 0.0,
  'stanza_ratio': 0.0},
 'Serge Abiteboul': {'number_of_spacy_entity': 0,
  'number_of_stanza entity': 0,
  'spacy_ratio': 0.0,
  'stanza_ratio': 0.0},
 'Samson Abramsky': {'number_of_spacy_entity': 0,
  'number_of_stanza entity': 0,
  'spacy_ratio': 0.0,
  'stanza_ratio': 0.0},
 'Leonard Adleman': {'number_of_spacy_entity': 0,
  'number_of_stanza entity': 0,
  'spacy_ratio': 0.0,
  